In [1]:
import numpy as np
import scipy.linalg as linalg
from numpy import exp

In [2]:
S0=40; K=40; r=0.1; T=1; sigma=0.3; Sbarrier=30; Smax=100; M=100; N=100; CALLvsPUT='PUT'

In [3]:
dS=(Smax-Sbarrier)/float(M)
dt=T/float(N)
boundary_conds=np.linspace(Sbarrier,Smax,M+1)
i_values = boundary_conds/dS
j_values=np.arange(N)
grid=np.zeros(shape=(M+1,N+1))

In [4]:
print('dS=',dS);print('dt=',dt);print('boundary_conds=',boundary_conds);print('i_values=',i_values)

dS= 0.7
dt= 0.01
boundary_conds= [ 30.   30.7  31.4  32.1  32.8  33.5  34.2  34.9  35.6  36.3  37.   37.7
  38.4  39.1  39.8  40.5  41.2  41.9  42.6  43.3  44.   44.7  45.4  46.1
  46.8  47.5  48.2  48.9  49.6  50.3  51.   51.7  52.4  53.1  53.8  54.5
  55.2  55.9  56.6  57.3  58.   58.7  59.4  60.1  60.8  61.5  62.2  62.9
  63.6  64.3  65.   65.7  66.4  67.1  67.8  68.5  69.2  69.9  70.6  71.3
  72.   72.7  73.4  74.1  74.8  75.5  76.2  76.9  77.6  78.3  79.   79.7
  80.4  81.1  81.8  82.5  83.2  83.9  84.6  85.3  86.   86.7  87.4  88.1
  88.8  89.5  90.2  90.9  91.6  92.3  93.   93.7  94.4  95.1  95.8  96.5
  97.2  97.9  98.6  99.3 100. ]
i_values= [ 42.85714286  43.85714286  44.85714286  45.85714286  46.85714286
  47.85714286  48.85714286  49.85714286  50.85714286  51.85714286
  52.85714286  53.85714286  54.85714286  55.85714286  56.85714286
  57.85714286  58.85714286  59.85714286  60.85714286  61.85714286
  62.85714286  63.85714286  64.85714286  65.85714286  66.85714286
  67.857142

In [9]:
if CALLvsPUT=='CALL':
    grid[:, -1]= np.maximum(boundary_conds-K,0)
    grid[-1, :-1]= Smax - K*exp(-r*dt*(N-j_values)) 
        
if CALLvsPUT=='PUT':
    grid[:, -1] = np.maximum(K-boundary_conds, 0)
    print(grid[:, -1])
    grid[0, :] = 0.0
    print(grid[0, :])
    print(grid[:, -1]) ## value changing from max(40-30,0) to zero bcoz of touching barrier

[10.   9.3  8.6  7.9  7.2  6.5  5.8  5.1  4.4  3.7  3.   2.3  1.6  0.9
  0.2  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
  0.   0.   0. ]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0.]
[0.  9.3 8.6 7.9 7.2 6.5 5.8 5.1 4.4 3.7 3.  2.3 1.6 0.9 0.2 0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.

In [6]:
a= 0.5*dt*((r*i_values) - (sigma**2 * i_values**2))
b= 1 + dt*((sigma**2 * i_values**2) + r)
c= -0.5*dt*((r*i_values) + (sigma**2 * i_values**2))

coeffs= np.diag(a[2:M],-1) + np.diag(b[1:M]) + np.diag(c[1:M-1],1)
P, L, U = linalg.lu(coeffs)
aux = np.zeros(M-1)
for j in reversed(range(N)):
    aux[0] = np.dot(a[1], grid[0, j])
    aux[-1]= np.dot(c[-1],grid[-1,j])
    x1 = linalg.solve(L, grid[1:M, j+1]-aux)
    x2 = linalg.solve(U, x1)
    grid[1:M, j] = x2            

In [7]:
np.interp(S0,boundary_conds,grid[:,0])

0.5913565603678127